In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/home/ivo/Programming_Personal_Projects/Cluster Task/clustering-problem/.ct_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')


In [3]:
data = pd.read_csv("../data/location_data.csv")

In [4]:
data.describe()

,city,road,house_number,postcode,state
count,63695,63695,63695,63695,63695
unique,5534,17184,6353,11015,121
top,houston,main st,101,46227,ca
freq,640,599,357,33,3251


In [5]:
data.head()

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl


In [6]:
sentences = data["road"]

In [7]:
sentence_embeddings = model.encode(sentences)
# this may not scale well...

In [8]:
sentence_embeddings.shape

(63695, 384)

In [25]:
X1 = sentence_embeddings.astype("float16").copy()
X2 = X1.transpose()


In [28]:
for row_id in range(X1.shape[1]):
    cosine_similarity(X1[row_id],)

SyntaxError: unexpected EOF while parsing (293489469.py, line 1)

In [28]:
X1.shape

(63695, 384)

In [31]:
K = np.exp(-np.sum((X1 - X2)**2, axis=-1))

MemoryError: Unable to allocate 5.67 TiB for an array with shape (63695, 63695, 384) and data type float32